In [1]:
%pwd 

'/workspaces/OpenDVCW/train_models'

In [2]:
# %cd /home/ubu-admin/Developer/tensorflow-wavelets
%cd /workspaces/OpenDVCW
import OpenDVCW
import numpy as np
import load
import tensorflow as tf
import matplotlib.pyplot as plt
import DataGen
import Callbacks
import datetime

/workspaces/OpenDVCW


In [3]:
BATCH_SIZE = 1
EPOCHS = 800
STEPS_PER_EPOCH = 100
Height = 240
Width = 240
Channel = 3
lmbda = 16384
lr_init = 1e-4
early_stop = 400
I_QP=27
wavelet_name = "haar"
args = OpenDVCW.Arguments()

timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkponts_last_path = ""
checkponts_new_path = "checkpoints_wavelets_{}_Lmbd_{}_epcs_{}_es_{}_I_QP_{}_{}x{}_CosineDecay_{}/".format(wavelet_name, lmbda, EPOCHS,  early_stop, I_QP, Width, Height, timestamp)
save_name = "model_save_" + checkponts_new_path


In [4]:
%load_ext tensorboard


In [5]:
# rm -rf ./logs/

In [6]:
log_dir = "logs/fit/" + timestamp

In [7]:
model = OpenDVCW.OpenDVC(width=Width, height=Height, batch_size=BATCH_SIZE, num_filters=128, lmbda=lmbda)

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=lr_init, decay_steps=EPOCHS*(STEPS_PER_EPOCH), alpha=1e-8, name="lr_CosineDecay")


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),)
print("* [Model compiled]...")

print("* [Loading dataset]...")
data = DataGen.DataVimeo90kGenerator("folder_cloud_test.npy", 
                                    BATCH_SIZE,
                                    (Height,Width,Channel),
                                    Channel,
                                    True, 
                                    I_QP,
                                    True)

print("Loading weights")
if not checkponts_last_path == "":
    model.load_weights(checkponts_last_path)


2022-04-28 04:36:54.941475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 04:36:55.028544: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 04:36:55.029719: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-28 04:36:55.032895: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

* [Model compiled]...
* [Loading dataset]...
Loading weights


In [8]:
model.summary()

Model: "open_dvc"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mv_analysis (AnalysisTransf  (1, 15, 15, 128)         659456    
 orm)                                                            
                                                                 
 mv_synthesis (SynthesisTran  (1, 240, 240, 2)         642824    
 sform)                                                          
                                                                 
 res_analysis (AnalysisTrans  (None, 15, 15, 128)      1552640   
 form)                                                           
                                                                 
 res_synthesis (SynthesisTra  (None, 240, 240, 3)      1536015   
 nsform)                                                         
                                                                 
 wavelets_optical_flow (Wave  multiple                 240

2022-04-28 04:37:12.237128: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [9]:
# model.layers[0].trainable = False
# model.layers[1].trainable = False
# model.layers[2].trainable = True
# model.layers[3].trainable = True
# model.layers[4].trainable = False
# model.layers[5].trainable = True

In [10]:
for layer in model.layers:
    print(layer.name, layer.trainable)

mv_analysis True
mv_synthesis True
res_analysis True
res_synthesis True
wavelets_optical_flow True
motion_compensation True


In [11]:
# tf.config.run_functions_eagerly(True)
hist = model.fit(x=data, steps_per_epoch=STEPS_PER_EPOCH, epochs=EPOCHS, verbose=1, batch_size=BATCH_SIZE,
                callbacks=[
                    # Callbacks.MemoryCallback(),
                    # Callbacks.LearningRateReducer(),
                    tf.keras.callbacks.ModelCheckpoint(filepath=checkponts_new_path, save_weights_only=True, save_freq='epoch', monitor="loss", mode='min',  save_best_only=True, verbose=1), 
                    tf.keras.callbacks.TerminateOnNaN(),
                    tf.keras.callbacks.EarlyStopping(monitor='loss', patience=early_stop),
                    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, update_freq="epoch"),            
                    ],
				)

Epoch 1/800


2022-04-28 04:38:03.140201: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


100/100 [==============================] - ETA: 0s - loss: 213.1486 - bpp: 5.3395 - mse: 0.0127
Epoch 1: loss improved from inf to 213.14859, saving model to checkpoints_wavelets_haar_Lmbd_16384_epcs_800_es_400_I_QP_27_240x240_CosineDecay_20220428-043653/
100/100 [==============================] - 88s 256ms/step - loss: 213.1486 - bpp: 5.3395 - mse: 0.0127
Epoch 2/800
100/100 [==============================] - ETA: 0s - loss: 57.1702 - bpp: 5.2694 - mse: 0.0032
Epoch 2: loss improved from 213.14859 to 57.17017, saving model to checkpoints_wavelets_haar_Lmbd_16384_epcs_800_es_400_I_QP_27_240x240_CosineDecay_20220428-043653/
100/100 [==============================] - 28s 281ms/step - loss: 57.1702 - bpp: 5.2694 - mse: 0.0032
Epoch 3/800
100/100 [==============================] - ETA: 0s - loss: 44.5982 - bpp: 5.2004 - mse: 0.0024
Epoch 3: loss improved from 57.17017 to 44.59825, saving model to checkpoints_wavelets_haar_Lmbd_16384_epcs_800_es_400_I_QP_27_240x240_CosineDecay_20220428-0436

In [12]:
path = load.load_random_path("folder_cloud_test.npy")
i=0
out_bin = "Test_com/test{}.bin".format(i)
out_decom = "Test_com/testdcom{}.png".format(i)
p_on_test = "Test_com/test_p_frame{}.png".format(i)
i_on_test = "Test_com/test_i_frame{}.png".format(i)

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)

OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))

OpenDVCW.compress(model, i_frame, p_frame, out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, out_bin, out_decom, 240, 240)

/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00070/0966/im1.png
compress
in the compress
decompress
in decompress


In [13]:
model.save(save_name, save_format="tf")

in the compress
in decompress


INFO:tensorflow:Assets written to: model_save_checkpoints_wavelets_haar_Lmbd_16384_epcs_800_es_400_I_QP_27_240x240_CosineDecay_20220428-043653/assets


INFO:tensorflow:Assets written to: model_save_checkpoints_wavelets_haar_Lmbd_16384_epcs_800_es_400_I_QP_27_240x240_CosineDecay_20220428-043653/assets


In [14]:
import cv2
import os
from math import log10, sqrt

def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


path = load.load_path_n("folder_cloud_test.npy", 0)
p_frame_out_bin = "Test_com/dvcw/p_frame_dvcw.bin"
out_decom = "Test_com/dvcw/frame1.png"
i_on_test = "Test_com/frame0.png"
p_on_test = "Test_com/frame1.png"

i_frame = path + 'im1' + '.png'
p_frame = path + 'im2' + '.png'
print(i_frame)
print(p_frame)

# write inputs to disk
OpenDVCW.write_png(p_on_test, OpenDVCW.read_png_crop(p_frame, 240, 240))
OpenDVCW.write_png(i_on_test, OpenDVCW.read_png_crop(i_frame, 240, 240))


OpenDVCW.compress(model, i_frame, p_frame, p_frame_out_bin, 240, 240)
OpenDVCW.decompress(model, i_frame, p_frame_out_bin, out_decom, 240, 240)


original = cv2.imread(p_on_test)
compressed = cv2.imread(out_decom)
bin_size = os.path.getsize(p_frame_out_bin)
value = PSNR(original, compressed)
print("bin size: ", bin_size , "psnr: ", value)


/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im1.png
/mnt/WindowsDev/DataSets/vimeo_septuplet/sequences/00049/0289/im2.png
compress
decompress
bin size:  7968 psnr:  42.49035184397427
